In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_Fatigue_Syndrome"
cohort = "GSE251792"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_Fatigue_Syndrome"
in_cohort_dir = "../../input/GEO/Chronic_Fatigue_Syndrome/GSE251792"

# Output paths
out_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/GSE251792.csv"
out_gene_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/gene_data/GSE251792.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/clinical_data/GSE251792.csv"
json_path = "../../output/preprocess/Chronic_Fatigue_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Deep phenotyping of Post-infectious Myalgic Encephalomyelitis/Chronic Fatigue Syndrome"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['Sex: Female', 'Sex: Male'], 1: ['age: 61', 'age: 37', 'age: 56', 'age: 24', 'age: 58', 'age: 43', 'age: 26', 'age: 40', 'age: 47', 'age: 22', 'age: 54', 'age: 44', 'age: 20', 'age: 23', 'age: 33', 'age: 25', 'age: 51', 'age: 48', 'age: 36', 'age: 38', 'age: 60', 'age: 50', 'age: 49', 'age: 55', 'age: 57'], 2: ['group: Patient', 'group: Control']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# Based on the series title and summary, this appears to be a SuperSeries on ME/CFS
# SuperSeries typically combine multiple datasets, but we need more information to determine
# if gene expression data is included. Since we don't have explicit confirmation,
# let's conservatively assume gene expression data is available
is_gene_available = True

# 2. Determine variable availability and create conversion functions

# 2.1 Identify rows containing trait, age, and gender data
trait_row = 2  # 'group: Patient', 'group: Control' indicates trait information
age_row = 1    # Contains age information
gender_row = 0  # Contains sex information

# 2.2 Create conversion functions for each variable

def convert_trait(value):
    """Convert trait values to binary format (1 for Patient, 0 for Control)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "patient":
        return 1
    elif value.lower() == "control":
        return 0
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numeric format"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to integer
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender values to binary format (0 for Female, 1 for Male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# 3. Save metadata for initial filtering
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Conduct initial filtering and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Create directory for clinical data if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Clinical features preview:", preview)
    
    # Save clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Clinical features preview: {'GSM7988184': [1.0, 61.0, 0.0], 'GSM7988185': [0.0, 37.0, 1.0], 'GSM7988186': [0.0, 56.0, 0.0], 'GSM7988187': [0.0, 56.0, 0.0], 'GSM7988188': [1.0, 24.0, 0.0], 'GSM7988189': [1.0, 58.0, 1.0], 'GSM7988190': [1.0, 43.0, 1.0], 'GSM7988191': [1.0, 26.0, 0.0], 'GSM7988192': [0.0, 40.0, 1.0], 'GSM7988193': [1.0, 47.0, 1.0], 'GSM7988194': [0.0, 22.0, 0.0], 'GSM7988195': [1.0, 54.0, 0.0], 'GSM7988196': [0.0, 58.0, 1.0], 'GSM7988197': [1.0, 44.0, 0.0], 'GSM7988198': [1.0, 20.0, 0.0], 'GSM7988199': [0.0, 26.0, 1.0], 'GSM7988200': [1.0, 23.0, 0.0], 'GSM7988201': [1.0, 33.0, 1.0], 'GSM7988202': [0.0, 54.0, 0.0], 'GSM7988203': [1.0, 25.0, 0.0], 'GSM7988204': [0.0, 58.0, 1.0], 'GSM7988205': [1.0, 37.0, 1.0], 'GSM7988206': [0.0, 23.0, 1.0], 'GSM7988207': [1.0, 22.0, 1.0], 'GSM7988208': [1.0, 51.0, 0.0], 'GSM7988209': [1.0, 48.0, 1.0], 'GSM7988210': [0.0, 36.0, 1.0], 'GSM7988211': [0.0, 56.0, 0.0], 'GSM7988212': [1.0, 38.0, 0.0], 'GSM7988213': [1.0, 60.0, 1.0], 'GSM7988214'

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 68
Header line: "ID_REF"	"GSM7988184"	"GSM7988185"	"GSM7988186"	"GSM7988187"	"GSM7988188"	"GSM7988189"	"GSM7988190"	"GSM7988191"	"GSM7988192"	"GSM7988193"	"GSM7988194"	"GSM7988195"	"GSM7988196"	"GSM7988197"	"GSM7988198"	"GSM7988199"	"GSM7988200"	"GSM7988201"	"GSM7988202"	"GSM7988203"	"GSM7988204"	"GSM7988205"	"GSM7988206"	"GSM7988207"	"GSM7988208"	"GSM7988209"	"GSM7988210"	"GSM7988211"	"GSM7988212"	"GSM7988213"	"GSM7988214"	"GSM7988215"	"GSM7988216"	"GSM7988217"	"GSM7988218"	"GSM7988219"	"GSM7988220"	"GSM7988221"	"GSM7988222"	"GSM7988223"	"GSM7988224"	"GSM7988225"	"GSM8032049"	"GSM8032050"	"GSM8032051"	"GSM8032052"	"GSM8032053"	"GSM8032054"	"GSM8032055"	"GSM8032056"	"GSM8032057"	"GSM8032058"	"GSM8032059"	"GSM8032060"	"GSM8032061"	"GSM8032062"	"GSM8032063"	"GSM8032064"	"GSM8032065"	"GSM8032066"	"GSM8032067"	"GSM8032068"	"GSM8032069"	"GSM8032070"	"GSM8032071"	"GSM8032072"	"GSM8032073"	"GSM8032074"	"GSM8032075"	"GSM8032076"	"GSM8032077"	"GSM8032078"	"GSM8032079"	

### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers in the expression data
# The identifiers with prefixes 'HCE' and 'SL' appear to be probe IDs 
# rather than standard human gene symbols (like BRCA1, TP53, etc.)
# These will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['SL019100', 'SL007136', 'SL001731', 'SL019096', 'SL005173'], 'TargetFullName': ['E3 ubiquitin-protein ligase CHIP', 'CCAAT/enhancer-binding protein beta', 'Gamma-enolase', 'E3 SUMO-protein ligase PIAS4', 'Interleukin-10 receptor subunit alpha'], 'Target': ['CHIP', 'CEBPB', 'NSE', 'PIAS4', 'IL-10 Ra'], 'PT_LIST': ['Q9UNE7', 'P17676', 'P09104', 'Q8N2W9', 'Q13651'], 'Entrez_GENE_ID_LIST': ['10273', '1051', '2026', '51588', '3587'], 'EntrezGeneSymbol': ['STUB1', 'CEBPB', 'ENO2', 'PIAS4', 'IL10RA'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the annotation contain probe IDs and gene symbols
id_column = 'ID'  # The column in gene_annotation that corresponds to the probe IDs in gene_data
gene_symbol_column = 'EntrezGeneSymbol'  # The column containing gene symbols

# 2. Create a mapping dataframe between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, id_column, gene_symbol_column)

# 3. Apply the mapping to convert probe-level measurements to gene-level expression
# This handles many-to-many mappings by dividing expression values equally among genes
# for each probe, then summing all contributions for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene expression data after mapping
print("Gene expression data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {len(gene_data.columns)}")
print("First few genes:")
print(gene_data.head())

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data after mapping:
Number of genes: 1315
Number of samples: 84
First few genes:
      GSM7988184  GSM7988185  GSM7988186  GSM7988187  GSM7988188  GSM7988189  \
Gene                                                                           
A2M       2645.2      1851.6      2049.7      3365.0      1905.3      2880.7   
ABL1        26.8        29.9        33.8        65.0        23.7        37.2   
ABL2        56.7        85.5        73.2        55.0        89.3        57.4   
ACAN       528.8       443.1       268.1       357.7       213.8       446.2   
ACE2        21.3        17.2        25.9        14.2        19.7        18.7   

      GSM7988190  GSM7988191  GSM7988192  GSM7988193  ...  GSM8032081  \
Gene                                                  ...               
A2M       2093.6      2079.1      2820.9      2171.8  ...     25789.1   
ABL1        33.3        31.1        21.1        26.7  ...       416.3   
ABL2        62.0        72.6        60.8        59

Gene expression data saved to ../../output/preprocess/Chronic_Fatigue_Syndrome/gene_data/GSE251792.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from monocytes of rheumatoid arthritis patients, with osteoporosis status included in comorbidity information."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Chronic_Fatigue_Syndrome/gene_data/GSE251792.csv


Clinical data saved to ../../output/preprocess/Chronic_Fatigue_Syndrome/clinical_data/GSE251792.csv
Linked data shape: (84, 1305)
For the feature 'Chronic_Fatigue_Syndrome', the least common label is '1.0' with 42 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Chronic_Fatigue_Syndrome' in this dataset is fine.

Quartiles for 'Age':
  25%: 33.0
  50% (Median): 47.0
  75%: 56.0
Min: 20.0
Max: 61.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 38 occurrences. This represents 45.24% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



A new JSON file was created at: ../../output/preprocess/Chronic_Fatigue_Syndrome/cohort_info.json
Linked data saved to ../../output/preprocess/Chronic_Fatigue_Syndrome/GSE251792.csv
